# EDA

## Import Packages

In [24]:
import pandas as pd
import seaborn as sns

## Import Data and merge csv files

In [2]:
import os, glob
import pandas as pd

path = "data-2"

all_files = glob.glob(os.path.join(path, "*.csv"))

all_df = []
for f in all_files:
    df = pd.read_csv(f, sep=',')
    df['file'] = f.split('/')[-1]
    all_df.append(df)
    
merged_df = pd.concat(all_df, ignore_index=True, sort=True)

## First look at the data

In [3]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209222 entries, 0 to 209221
Data columns (total 38 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   backers_count             209222 non-null  int64  
 1   blurb                     209214 non-null  object 
 2   category                  209222 non-null  object 
 3   converted_pledged_amount  209222 non-null  int64  
 4   country                   209222 non-null  object 
 5   created_at                209222 non-null  int64  
 6   creator                   209222 non-null  object 
 7   currency                  209222 non-null  object 
 8   currency_symbol           209222 non-null  object 
 9   currency_trailing_code    209222 non-null  bool   
 10  current_currency          209222 non-null  object 
 11  deadline                  209222 non-null  int64  
 12  disable_communication     209222 non-null  bool   
 13  file                      209222 non-null  o

In [5]:
compression_opts = dict(method='zip', archive_name='out.csv')  
merged_df.to_csv('out.zip', index=False, compression=compression_opts)

In [7]:
merged_df.shape

(209222, 38)

In [8]:
merged_df.head()

,backers_count,blurb,category,converted_pledged_amount,country,created_at,creator,currency,currency_symbol,currency_trailing_code,...,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type
0,315,Babalus Shoes,"{""id"":266,""name"":""Footwear"",""slug"":""fashion/fo...",28645,US,1541459205,"{""id"":2094277840,""name"":""Lucy Conroy"",""slug"":""...",USD,$,True,...,babalus-childrens-shoes,https://www.kickstarter.com/discover/categorie...,False,False,live,1548223375,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",28645.000000,international
1,47,A colorful Dia de los Muertos themed oracle de...,"{""id"":273,""name"":""Playing Cards"",""slug"":""games...",1950,US,1501684093,"{""id"":723886115,""name"":""Lisa Vollrath"",""slug"":...",USD,$,True,...,the-ofrenda-oracle-deck,https://www.kickstarter.com/discover/categorie...,True,False,successful,1504976459,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",1950.000000,domestic
2,271,"Electra's long awaited, eclectic Debut Pop/Roc...","{""id"":43,""name"":""Rock"",""slug"":""music/rock"",""po...",22404,US,1348987533,"{""id"":323849677,""name"":""Electra"",""is_registere...",USD,$,True,...,record-electras-debut-album-pop-rock-classical,https://www.kickstarter.com/discover/categorie...,True,False,successful,1371013395,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",22404.000000,international
3,3,The Mist of Tribunal is a turn-based card game...,"{""id"":273,""name"":""Playing Cards"",""slug"":""games...",165,GB,1483780271,"{""id"":196281496,""name"":""Artur Ordijanc (delete...",GBP,£,False,...,the-mist-of-tribunal-a-card-game,https://www.kickstarter.com/discover/categorie...,False,False,failed,1489425776,1.216066,"{""web"":{""project"":""https://www.kickstarter.com...",165.384934,domestic
4,3,"Livng with a brain impairment, what its like t...","{""id"":48,""name"":""Nonfiction"",""slug"":""publishin...",2820,US,1354817071,"{""id"":1178460181,""name"":""Dawn Johnston"",""is_re...",USD,$,True,...,help-change-the-face-of-brain-impairment,https://www.kickstarter.com/discover/categorie...,True,False,successful,1357763527,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",2820.000000,domestic


In [9]:
merged_df.columns

Index(['backers_count', 'blurb', 'category', 'converted_pledged_amount',
       'country', 'created_at', 'creator', 'currency', 'currency_symbol',
       'currency_trailing_code', 'current_currency', 'deadline',
       'disable_communication', 'file', 'friends', 'fx_rate', 'goal', 'id',
       'is_backing', 'is_starrable', 'is_starred', 'launched_at', 'location',
       'name', 'permissions', 'photo', 'pledged', 'profile', 'slug',
       'source_url', 'spotlight', 'staff_pick', 'state', 'state_changed_at',
       'static_usd_rate', 'urls', 'usd_pledged', 'usd_type'],
      dtype='object')

In [10]:
merged_df.groupby('state').count()

,backers_count,blurb,category,converted_pledged_amount,country,created_at,creator,currency,currency_symbol,currency_trailing_code,...,profile,slug,source_url,spotlight,staff_pick,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type
state,,,,,,,,,,,,,,,,,,,,,
canceled,8624,8618,8624,8624,8624,8624,8624,8624,8624,8624,...,8624,8624,8624,8624,8624,8624,8624,8624,8624,8624
failed,75199,75197,75199,75199,75199,75199,75199,75199,75199,75199,...,75199,75199,75199,75199,75199,75199,75199,75199,75199,75199
live,7311,7311,7311,7311,7311,7311,7311,7311,7311,7311,...,7311,7311,7311,7311,7311,7311,7311,7311,7311,6869
successful,117465,117465,117465,117465,117465,117465,117465,117465,117465,117465,...,117465,117465,117465,117465,117465,117465,117465,117465,117465,117427
suspended,623,623,623,623,623,623,623,623,623,623,...,623,623,623,623,623,623,623,623,623,623


In [14]:
pd.isnull(merged_df).sum()

backers_count                    0
blurb                            8
category                         0
converted_pledged_amount         0
country                          0
created_at                       0
creator                          0
currency                         0
currency_symbol                  0
currency_trailing_code           0
current_currency                 0
deadline                         0
disable_communication            0
file                             0
friends                     208922
fx_rate                          0
goal                             0
id                               0
is_backing                  208922
is_starrable                     0
is_starred                  208922
launched_at                      0
location                       226
name                             0
permissions                 208922
photo                            0
pledged                          0
profile                          0
slug                

## Data Cleaning

In [108]:
final = merged_df.copy()

### Dropping Data

#### dropping variables with missing values

In [109]:
final = final.drop(['blurb', 'friends', 'is_backing', 'is_starred', 'permissions', 'usd_type', 'location'], axis = 1)

#### dropping useless variables

In [110]:
final = final.drop(['creator', 'currency_symbol', 'name', 'photo', 'profile', 'slug', 'source_url', 'urls', 'pledged', 'usd_pledged'], axis = 1)

#### dropping file column 

In [111]:
final = final.drop(['file'], axis = 1)

In [112]:
pd.isnull(final).sum()

backers_count               0
category                    0
converted_pledged_amount    0
country                     0
created_at                  0
currency                    0
currency_trailing_code      0
current_currency            0
deadline                    0
disable_communication       0
fx_rate                     0
goal                        0
id                          0
is_starrable                0
launched_at                 0
spotlight                   0
staff_pick                  0
state                       0
state_changed_at            0
static_usd_rate             0
usd_pledged                 0
dtype: int64

### Time conversions

In [94]:
final.launched_at = pd.to_datetime(final.launched_at,unit='s',infer_datetime_format=True)
final.deadline = pd.to_datetime(final.deadline,unit='s',infer_datetime_format=True)
final.created_at = pd.to_datetime(final.created_at,unit='s',infer_datetime_format=True)
final.state_changed_at = pd.to_datetime(final.state_changed_at,unit='s',infer_datetime_format=True)

### creating new variables

In [122]:
final['pledged_over'] = final.converted_pledged_amount - final.goal
final['campaign_days'] = final.deadline - final.launched_at 
final['pre_launched_days'] = final.launched_at - final.created_at

In [123]:
final.pledged_over.mean()

-36283.13077611341

In [42]:
final.campaign_days.mean()

Timedelta('32 days 17:22:26.681759088')

In [61]:
final.pre_launched_days.mean()

Timedelta('47 days 15:36:24.747727294')

# category exploration

In [131]:
# To do: save final df as csv
#compression_opts = dict(method='zip', archive_name='Kickstarter_all_clean.csv')  
#final.to_csv('Kickstarter_all_clean.zip', index=False, compression=compression_opts)